<a href="https://colab.research.google.com/github/nverchev/other_Python_projects/blob/main/Chamfer_distance_for_Pytorch_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is the best implementation of the Chamfer Distance?

We compare a simple implementation with other two coming from two libraries.
- https://github.com/otaheri/chamfer_distance
- https://pypi.org/project/chamferdist/1.0.0/#description

These implementatations are compatible with torch.autograd, and we test them using GPU power.

I suggest to run the experiment different times to account for some possible overhead.

In [1]:
#First implementation 
!pip install git+'https://github.com/otaheri/chamfer_distance'

#Second implementation
!pip install chamferdist


  Cloning https://github.com/otaheri/chamfer_distance to /tmp/pip-req-build-339r55yu
  Running command git clone -q https://github.com/otaheri/chamfer_distance /tmp/pip-req-build-339r55yu
     |████████████████████████████████| 108 kB 5.8 MB/s 
  Created wheel for chamfer-distance: filename=chamfer_distance-0.1-py3-none-any.whl size=5653 sha256=bc2fd7cf40db8d70b9ed35bded402d47689e6bd608b4b15d8b6b23a05f6c1744
  Stored in directory: /tmp/pip-ephem-wheel-cache-gz8keqk8/wheels/2a/c5/7c/395771526a57f81590f5b9e2be57f219f834d894e10b1cd993
Successfully built chamfer-distance
  Created wheel for chamferdist: filename=chamferdist-1.0.0-cp37-cp37m-linux_x86_64.whl size=5836539 sha256=75262b5ebc6eaf550ac4c973b50649f2c22466133013c26fbd9389553f9caa71
  Stored in directory: /root/.cache/pip/wheels/28/bb/d1/c789ecd6835e466e813f6e2c5e23bb1bbb2248e84586ba82d2
Successfully built chamferdist


In [7]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

pc1 = torch.rand([100,1000,3]).to()
pc2 = torch.rand([100,100,3]).to()

In [14]:
from chamfer_distance import ChamferDistance
chamdist = ChamferDistance()
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
d1,d2,_,_ = chamdist(pc1,pc2) # dist forward, dist reverse
d = d1.sum(axis=1).mean() + d2.sum(axis=1).mean() # batchmean
end.record()
torch.cuda.synchronize()

print("First implementation:")
print("Time (ms): ", start.elapsed_time(end))
print("Result: ", d)

First implementation:
Time (ms):  205.4356231689453
Result:  tensor(19.5597)


In [15]:
from chamferdist import ChamferDistance
chamdist = ChamferDistance()
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
d = chamdist(pc1,pc2, bidirectional=True, reduction="mean") 
end.record()
torch.cuda.synchronize()

print("Second implementation:")
print("Time (ms): ", start.elapsed_time(end))
print("Result: ", d)

Second implementation:
Time (ms):  198.42588806152344
Result:  tensor(19.5597)


In [16]:
def square_distance(t1, t2):
    t2 = t2.permute(0, 2, 1)
    dist = -2 * torch.matmul(t1, t2)
    dist += torch.sum(t1 ** 2, 2, keepdim=True)
    dist += torch.sum(t2 ** 2, 1, keepdim=True)
    return dist
    
# Chamfer Distance
def chamdist(t1, t2): 
    dist = square_distance(t1, t2)
    # forward + reverse
    return torch.min(dist, axis = 2)[0].mean(0).sum()\
         + torch.min(dist, axis = 1)[0].mean(0).sum()

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
d = chamdist(pc1, pc2)
end.record()
torch.cuda.synchronize()

print("Simple implementation:")
print("Time: ", start.elapsed_time(end))
print("Result: ", d)

Simple implementation:
Time:  72.68150329589844
Result:  tensor(19.5597)


# Discussion
Surprisingly, the simple implementation in torch is quicker than the ones from the two libraries.

Hope this could be useful to anybody starting with PCD.
